On va récupérer dans la page Wikipédia de la liste des gares RER les correspondances de tram, métro et RER qui passent par une gare donnée afin de les stocker dans un dictionnaire correspondant.


In [1]:
import requests
from bs4 import BeautifulSoup

# URL de la page Wikipédia
url = 'https://fr.wikipedia.org/wiki/Liste_des_gares_du_RER_d%27%C3%8Ele-de-France'

# Récupérer le contenu de la page
response = requests.get(url)
html = response.content

# Créer l'objet BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

def clean_text(text):
    # Nettoie le texte en remplaçant les caractères spéciaux.
    return text.replace('\xa0', ' ').strip() 

# Dictionnaire pour stocker les données
lignes = []
gares = []  
    
# Recherche des lignes de tableau (tr) contenant les données
for row in soup.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 1:
        # Extraire le nom de la station
        station_link = cells[0].find('a')
        if station_link:
            station_name = clean_text(station_link.get_text())
            gares.append(station_name)

            # Extraire les lignes de correspondance
            lignes.append([clean_text(link['title'].replace("Ligne", "")) for cell in cells[1:] for link in cell.find_all('a', title=True) if 'Ligne' in link['title']])
        


On a désormais deux listes de listes, qui contiennent le nom des gares ainsi que les lignes de RER/métro/Transilien qui y passent. On crée alors "url_gare", une liste contenant les adresses web menant vers les pages des gares correspondantes. On peut alors faire une boucle pour aller récupérer dans chaque page les données sur la circulation des bus.

In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://fr.wikipedia.org/wiki/Liste_des_gares_du_RER_d%27%C3%8Ele-de-France"

# Faire une requête GET à l'URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
    
    # Trouver le tableau contenant les informations sur les gares du RER
tableau_gares = soup.find("table", class_=["wikitable sortable center"])
tableau_gares_non_desservies = soup.find("table", class_=["wikitable sortable"])

url_gares=[]

    # Parcourir les lignes du tableau pour obtenir les liens dans la première colonne
for row in tableau_gares.find_all("tr")[1:]:  # Ignorer la première ligne (en-têtes)
        cellules = row.find_all("td")
        lien = cellules[0].find("a")
        url_gare = "https://fr.wikipedia.org" + lien.get("href")
        url_gares.append(url_gare)

for row in tableau_gares_non_desservies.find_all("tr")[1:]:  # Ignorer la première ligne (en-têtes)
        cellules = row.find_all("td")
        lien = cellules[0].find("a")
        url_gare = "https://fr.wikipedia.org" + lien.get("href")
        url_gares.append(url_gare)
                
bus=[]
bus_jour = []
noctilien = []

for url in url_gares :
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    bandeau = soup.find("table", {'class' : 'infobox_v2 noarchive'})
    td_tags = bandeau.find_all("td")
# Initialiser une liste pour stocker les numéros de ligne de bus
    numeros_lignes_bus = []

    for td_tag in td_tags : 
    # Trouver toutes les balises <b> à l'intérieur de la balise <td>
        balises_b = td_tag.find_all("b")

    # Extraire les numéros de ligne de bus des balises <b>
        for balise_b in balises_b:
            numero_ligne = balise_b.get_text()
            numeros_lignes_bus.append(clean_text(numero_ligne))
            
    # On garde uniquement les numéros des lignes de bus en excluant le nom de la ligne pour puvoir les compter facilement, puis on sépare les lignes d ejour et les noctiliens
    bus_clean = [num for num in numeros_lignes_bus if (any(i.isdigit() for i in num) or len(num)<3)]
    nocti = [i for i in bus_clean if i.startswith('N') and i[1].isdigit()]
    bus_j = [x for x in bus_clean if not (x.startswith('N') and x[1].isdigit())]
    concat = [bus_jour,noctilien]
    bus.append(concat)
    bus_jour.append(bus_j)
    noctilien.append(nocti)
    
                

On obtient en sortie, dans les listes bus_jour et noctilien, le nom des bus qui passent par chaque gare RER. L'intérêt est alors de les compter, et de regarder si cela a une influence sur la fréquentation desdites stations.

In [3]:
import pandas as pd 

for i in range(len(gares)):
    gares[i] = gares[i].upper()
    
concat = { 
    'arret' : gares,
    'lignes' : lignes,
    'bus_jour' : bus_jour,
    'noctilien' : noctilien,
}

data_bus = pd.DataFrame(concat)

quanti_bus = data_bus.copy()
quanti_bus['lignes'] = quanti_bus['lignes'].apply(lambda x: len(x))
quanti_bus['bus_jour'] = quanti_bus['bus_jour'].apply(lambda x: len(x))
quanti_bus['noctilien'] = quanti_bus['noctilien'].apply(lambda x: len(x))
quanti_bus

,arret,lignes,bus_jour,noctilien
0,CERGY-LE-HAUT,2,12,2
1,CERGY-SAINT-CHRISTOPHE,2,16,2
2,CERGY-PRÉFECTURE,2,42,2
3,NEUVILLE-UNIVERSITÉ,2,7,1
4,CONFLANS-FIN-D'OISE,3,9,2
...,...,...,...,...
244,IGNY,2,3,0
245,LONGJUMEAU,2,5,1
246,CHILLY-MAZARIN,2,3,1
247,GRAVIGNY-BALIZY,2,3,0


Ce dataframe final nous donne le nombre de ligne qui passe par chaque station.